# Outline

* array of acceptable genres
* Input: 10 games -->

* all movies w/in these genres, w/in accepted
* --> console as filer --> genres as filter, queried data - keywords mlo

* query: {name:keywords}

kelly will do this using postman

* data in: name, keywords

* process: calculate dist as # common keywords, to keywords in games user chose

* output: closest games to target keywords

# Code

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
pd.options.display.max_rows = 5
from sklearn.feature_extraction import DictVectorizer
from sklearn.neighbors import KNeighborsRegressor

games = pd.read_csv("IGDB_games.csv", sep=",")
games.head()

*For working on User.py, will move to file eventually*

In [ ]:
class User:
    def __init__(self, gn, g, t):  # self, game_names, genres, themes):
        self.name = "TESTerrr"
        self.game_names = pd.DataFrame(gn, columns=['name'])
        self.genres = pd.DataFrame(g, columns=['name'])
        self.themes = pd.DataFrame(t, columns=['name'])

# '''    def printUser(self):
#         print(self.name, "'s profile:\n- ", self.game_names,
#               "\n- ", self.genres, "\n-", self.themes)'''
    # def getIds(self):



In [ ]:
'''USER UPPERCASE AS FIRST LETTER'''
gn, g, t = ["Skyrim","Destiny","Mario Kart","Forza"],["Adventure","Racing","Shooter"],["Action","Survival"]
new_user = User(gn, g, t)


In [ ]:
# to be getIDs
genre_key = pd.read_csv("../DataExtraction/ids_with_genres.csv", sep=",")
new_user.genres = new_user.genres.merge(genre_key, how="left", on="name")
theme_key = pd.read_csv("../DataExtraction/themes_with_ids.csv", sep=",")
new_user.themes = new_user.themes.merge(theme_key, how="left", on="name")


<h1> example with fake data </h1>

 EX:
 `{'name': ['Skyrim', 'Destiny', 'Mario Kart', 'Forza'], 'adventure': [1, 0,0,0], 'racing': [0, 0,1,1], 'shooter': [0, 1,0,0]}`


##### just use genre and themes for now, like we talked about

In [ ]:
#def removeNans(array1):
    # https://www.kite.com/python/answers/how-to-remove-nan-values-from-a-numpy-array-in-python
    # nan_array = np.isnan(array1)
    # not_nan_array = ~ nan_array
    # array2 = array1[not_nan_array]
    # return array2


In [ ]:
games_nona = games[(~games.name.isna()) & (~games.genres.isna()) & (~games.themes.isna())]
# to match shapes
#games_nona[['genres', 'themes']].dtypes
gdf = games_nona[['genres', 'themes']]

# gdf.dtypes
## later on need to make these categorical


In [ ]:
# Features in our model
features = ["genres", "themes"]
 # train on games dataset
X_train_dict = games_nona[features].to_dict(orient="records")
# get features from User
X_new_dict = {
    "genres": "30", #str(new_user.genres["id"][0]), # starting with one value
    "themes": "20", #str(new_user.themes["id"][0])
}
Y_train = games_nona["name"]
Y_new = new_user.game_names

# Dummy encoding
vec = DictVectorizer(sparse=False)
vec.fit(X_train_dict)
X_train = vec.transform(X_train_dict)
X_new = vec.transform(X_new_dict)
# skip scaling
# K-Nearest Neighbors Model
model = KNeighborsRegressor(n_neighbors=3)
model.fit(X_train, Y_train) # to make x 1d?
# # Scikit-Learn expects 2-dimensional arrays, so we need to 
# # turn the Series into a DataFrame with 1 row.
# X_new = X_new.to_frame().T

'''model.predict(X_new)'''



### Other draft:
Discarded draft using more than `keyword` and `genre`. Will use later.

In [ ]:
# %matplotlib inline
#!pip3 install openpyxl  # package to open xlsx files
#!pip3 install xlrd
#!pip3 install sklearn
import matplotlib.pyplot as plt
from scraping_igdb import get_token
from User import User
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
'''implement classification alg - KNN classifier'''
# will need code to translate game's name to id

features = ["name", "id", "involved_companies",
            "platforms", "key_words", "similar_games"]

# Get user name and ratings
new_user = User()
new_user.printUser()
userFeature = new_user.series

# open excel file --> EXCEL READING NOT WORKING?? ## allGames = pd.read_excel("IGDB_games_beta.xlsx")
allGames = pd.read_csv("IGDB_games_beta.csv")
allGames.columns = ["id", "name", "category", "genres", "themes",
                    "involved_companies", "platforms", "key_words", "similar_games", "release_dates", ]
# allGames.reindex("")
print(allGames)
# .head() wont work # https://stackoverflow.com/questions/62624980/vscode-python-pandas-dataframe-intellisense-doesnt-show-attributes-method

# Borrowed from CMPS3160 Lab08+09
X_train_dict = allGames[features].to_dict(orient="records")
X_new_dict = [{
    'name': new_user.name,
    # 'category': new_user.categories,
    'genres': new_user.genres,
    'consoles': new_user.consoles,
    'key_words': new_user.key_words,
}]
y_train = allGames["similar_games"]

# sci-kit preprocessing
vec = DictVectorizer(sparse=False)  # training dict --> vector
vec.fit(X_train_dict)
X_train = vec.transform(X_train_dict)
X_new = vec.transform(X_new_dict)

scaler = StandardScaler()  # scale training data
scaler.fit(X_train)
X_train_sc = scaler.transform(X_train)
X_new_sc = scaler.transform(X_new)

# Fit a 10-nearest neighbors model.
model = KNeighborsRegressor(n_neighbors=10)
model.fit(X_train_sc, y_train)
# Calculate the model predictions on the training data.
y_train_pred = model.predict(X_new_sc)
print("Prediction", y_train_pred)


'''# scraping IGDB
headers = get_token()
query = requests.post('https://api.igdb.com/v4/games', headers=headers)
'''